In [ ]:
import sys
sys.path.append('CLIP-dissect') 


import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import cv2
import glob
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import matplotlib
import similarity
import utils
import data_utils
from clip_dissect_pipeline import dissect_pipeline
%matplotlib inline

In [ ]:
dataset_path = "FER2013"

transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((48,48))])
train_dataset = ImageFolder(dataset_path+'/train',transform)
train_loader = DataLoader(dataset=train_dataset,batch_size=100*6)
#creating val data loaders
val_dataset = ImageFolder(dataset_path+'/test',transform)
val_loader = DataLoader(dataset=val_dataset,batch_size=100)

clip_name = 'ViT-B/16'
d_probe = 'FER2013'
concept_set = 'CLIP-dissect/data/concept_set.txt'
batch_size = 200
device = 'cuda'
pool_mode = 'avg'
similarity_fn = similarity.soft_wpmi

In [ ]:
import models
model_path = 'face_emotion_deep_emotion.pth'
target_model = models.Deep_Emotion().to('cuda')
target_model.load_state_dict(torch.load(model_path))

In [ ]:
pred, neuron_image_indices = dissect_pipeline(d_probe,concept_set, similarity_fn)

In [ ]:
import torch
import os
import random
import utils
import sys
import pytorchcv
import data_utils1

import cbm
import json

In [ ]:

load_dir = "Label-free-CBM/saved_model_jason_concept/saved_models/cifar100_lf_cbm"
device = "cuda"

model = cbm.load_cbm(load_dir, device)
model

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Update the dataset transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224, 224), interpolation=transforms.InterpolationMode.BILINEAR), # Adjusting input size
    # Add normalization here if required, e.g., transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_dataset = ImageFolder(root='FER2013/test', transform=transform)

# Update the model's final layer to match the number of classes in your dataset
num_classes = len(val_dataset.classes)
model.proj_layer = torch.nn.Linear(in_features=7178, out_features=num_classes, bias=False) 


In [ ]:
model.final = torch.nn.Linear(in_features=7, out_features=num_classes, bias=True) 

In [ ]:
classes = os.listdir(dataset_path+"/train")

In [ ]:
concept_set_path = 'CLIP-dissect/data/concept_set.txt'
with open(concept_set_path, 'r') as file:  
    concepts = file.read().split("\n")

In [ ]:
W_g = torch.load(os.path.join(load_dir, "W_g.pt"), map_location=device)
print("Before adjustment:", W_g)  # Check if it's all zeros

# Manually adjust a weight for debugging (this is just an example, adjust according to your actual dimensions)
W_g[0][0] = 1.0

# Load into the model
model.final.weight.data = W_g
print("After adjustment:", model.final.weight.data)

In [ ]:
to_show = random.choices([i for i in range(len(classes))], k=1)

for i in to_show:
    print("Output class:{} - {}".format(i, classes[i]))
    print("Incoming weights:")
    for j in range(len(concepts)):
        #if torch.abs(model.final.weight[i,j])>0.05:
            print("{} [{:.4f}] {}".format(concepts[j], model.final.weight[i,j], classes[i]))